In [2]:
import pandas as pd
from datetime import date, timedelta
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()

# SMA

In [3]:
def SMA(trend,slot,ndate):
    mv = 0
    sdate = date(2021,10,1)
    before = ndate - sdate
    before = int(before.total_seconds()/(60*60*24)) + 1

    if before >= slot:
        data = []
        for i in range(slot):
            temp_date = ndate - timedelta(i)
            temp_date = '_'.join(str(temp_date).split('-')) 
            temp_trend = pd.read_json('../assets/Result/newmm/'+str(temp_date)+'.json',encoding="utf8")
            temp_trend = temp_trend[temp_trend['LCS'] == trend]
            temp_trend.drop('total_match',axis='columns',inplace = True)

            if len(temp_trend) != 0:
                data.append(temp_trend.iloc[0]["match"])
            else:
                data.append(0)
        
        mv = round(sum(data)/slot, 2)
    return mv

In [ ]:
month = 12
day = 31

# for i in tqdm(range(1,5)):
#     df = pd.read_json(f"../assets/Result/newmm/2021_{month:02}_{i:02}.json", encoding="utf8")
#     df.rename(columns = {'match': 'frequency'}, inplace = True)
#     df["SMA"] = [0] * len(df)
#     df["dif"] = df["frequency"]
#     df.to_json(f"../assets/Result/MA/SMA/MINUS/2021_{month:02}_{i:02}.json", orient="records")
#     df.to_json(f"../assets/Result/MA/SMA/DIVIDE/2021_{month:02}_{i:02}.json", orient="records")

for i in tqdm(range(1,day+1)):
    df = pd.read_json(f"../assets/Result/newmm/2021_{month:02}_{i:02}.json", encoding="utf8")
    df.rename(columns = {'match': 'frequency'}, inplace = True)
    mv = []
    for lcs in df["LCS"]:
        avg = SMA(lcs, 5, date(2021,month,i))
        mv.append(avg)
    df["SMA"] = mv

    array1 = np.array(list(df["frequency"]))
    array2 = np.array(mv)

    subtracted_array = np.subtract(array1, array2)
    subtracted = list(subtracted_array)

    df["dif"] = subtracted
    df.sort_values(['dif'], ascending=False, inplace=True, ignore_index=True)
    df.to_json(f"../assets/Result/MA/SMA/MINUS/2021_{month:02}_{i:02}.json", orient="records")


    divided_array = np.divide(array1, array2)
    divided_array = np.around(divided_array, decimals=2)
    divided = list(divided_array)

    df["dif"] = divided
    df.sort_values(['dif'], ascending=False, inplace=True, ignore_index=True)
    df.to_json(f"../assets/Result/MA/SMA/DIVIDE/2021_{month:02}_{i:02}.json", orient="records")

# EMA

In [4]:
def EMA(trend,slot,ndate):
    mv = 0
    sdate = date(2021,10,slot)
    before = ndate - sdate
    before = int(before.total_seconds()/(60*60*24)) + 1

    if before > 0:
        for i in range(before):
            start = sdate + timedelta(i)
            # First Round Of EMA = Calculate SMA
            if i == 0:
                mv = SMA(trend,slot,start)
            # After that, Calculate with EMA formula
            else:
                temp_date = '_'.join(str(start).split('-'))
                temp_trend = pd.read_json('../assets/Result/newmm/'+str(temp_date)+'.json',encoding="utf8")
                temp_trend = temp_trend[temp_trend['LCS'] == trend]
                temp_trend.drop('total_match',axis='columns',inplace = True)

                if len(temp_trend) != 0:
                    v = temp_trend.iloc[0]["match"]
                    mv = (v - mv) * (2 / (1 + slot)) + mv
                else:
                    mv = (0 - mv) * (2 / (1 + slot)) + mv
    elif before == 0:
        mv = SMA(trend,slot,ndate)
    return round(mv, 2)

In [8]:
month = 12
day = 31

# for i in tqdm(range(1,5)):
#     df = pd.read_json(f"../assets/Result/newmm/2021_{month:02}_{i:02}.json", encoding="utf8")
#     df.rename(columns = {'match': 'frequency'}, inplace = True)
#     df["EMA"] = [0] * len(df)
#     df["dif"] = df["frequency"]
#     df.to_json(f"../assets/Result/MA/EMA/MINUS/2021_{month:02}_{i:02}.json", orient="records")
#     df.to_json(f"../assets/Result/MA/EMA/DIVIDE/2021_{month:02}_{i:02}.json", orient="records")

for i in tqdm(range(1,day+1)):
    df = pd.read_json(f"../assets/Result/newmm/2021_{month:02}_{i:02}.json", encoding="utf8")
    df.rename(columns = {'match': 'frequency'}, inplace = True)
    mv = []
    for lcs in df["LCS"]:
        avg = EMA(lcs, 5, date(2021,month,i))
        mv.append(avg)
    df["EMA"] = mv

    array1 = np.array(list(df["frequency"]))
    array2 = np.array(mv)

    subtracted_array = np.subtract(array1, array2)
    subtracted = list(subtracted_array)

    df["dif"] = subtracted
    df.sort_values(['dif'], ascending=False, inplace=True, ignore_index=True)
    df.to_json(f"../assets/Result/MA/EMA/MINUS/2021_{month:02}_{i:02}.json", orient="records")


    divided_array = np.divide(array1, array2)
    divided_array = np.around(divided_array, decimals=2)
    divided = list(divided_array)

    df["dif"] = divided
    df.sort_values(['dif'], ascending=False, inplace=True, ignore_index=True)
    df.to_json(f"../assets/Result/MA/EMA/DIVIDE/2021_{month:02}_{i:02}.json", orient="records")

100%|██████████| 31/31 [2:03:57<00:00, 239.93s/it]  


# Visualize

In [52]:
import pandas as pd

month, day = 10, 30
df = pd.read_json(f"../assets/Result/MA/EMA/MINUS/2021_{month:02}_{day:02}.json", encoding="utf8")
# df.sort_values(['dif','frequency'], ascending=False, inplace=True, ignore_index=True)
df.head(20)

,LCS,frequency,total_match,EMA,dif
0,ลอยกระทงสืบสานลอยกระทงทรงคุณค่าCOVID-FreeSetti...,22,695,7.33,14.67
1,CovidFreeSetting,17,695,5.68,11.32
2,ทางบกทางน้ำลอยกระทง,16,695,5.33,10.67
3,ภาพประกอบ[emailprotected],15,695,5.51,9.49
4,"""โจโจ้ไมอ๊อคลงสมัคร",14,695,4.67,9.33
5,วัลย์วณฺโณ,14,695,5.78,8.22
6,ภาพประกอบ[emailprotected,11,695,3.95,7.05
7,ปราโมทย์ปาทานโอ๊ตปราโมทย์มารุตบูรณ์ดีเจอาร์ต,10,695,3.33,6.67
8,สถานบันเทิงผับคาราโอเกะ,10,695,3.38,6.62
9,วัลย์ลาสิกขาสุธีสร้อยทองโอ๊ตปราโมทย์,8,695,2.67,5.33


In [51]:
df = pd.read_json(f"../assets/Result/newmm/2021_{month:02}_{day:02}.json", encoding="utf8")
df.head(20)

,LCS,match,total_match
0,พานแท้ชินวัตร,30,999
1,จิรภูริช.,28,999
2,ศรีสวัสดิ์ผู้บัญชาการทหาร,20,999
3,CREDITPHOTO[emailprotected,15,999
4,youtubewatchv,15,999
5,บุตรสาวชินวัตรคณะที่ปรึกษา,14,999
6,"ฤดูหนาวฤดูหนาวระดับความสูงความสูง3,500",11,999
7,เปิดเรียนOnsite,11,999
8,พั้นช์กาญจน์โร่ไหว้พระแก้บนสนามหลวง,10,999
9,พานแท้ชินวัตรบุตรชายชินวัตร,10,999
